This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Find the lowest common ancestor in a binary tree.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Is this a binary search tree?
    * No
* Can we assume the two nodes are in the tree?
    * No
* Can we assume this fits memory?
    * Yes

## Test Cases

<pre>
          _10_
        /      \
       5        9
      / \      / \
     12  3    18  20
        / \       /
       1   8     40
</pre>

* 0, 5 -> None
* 5, 0 -> None
* 1, 8 -> 3
* 12, 8 -> 5
* 12, 40 -> 10
* 9, 20 -> 9
* 3, 5 -> 5

## Algorithm

Refer to the [Solution Notebook]().  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

## Code

In [86]:
class Node(object):

    def __init__(self, key, left=None, right=None):
        self.key = key
        self.left = left
        self.right = right

    def __repr__(self):
        return str(self.key)

In [87]:
class LCAResult(object):
    def __init__(self, node, isAncestor = False):
        self.node = node
        self.isAncestor = isAncestor
    
class BinaryTree(object):
    
    def lca(self, root, node1, node2):
        return self.lcaWithoutCheckNodeInTree(root, node1, node2)
    
    def lcaWithCheckNodeInTree(self, root, node1, node2):
        if self._isNodeInTree(root, node1) and self._isNodeInTree(root, node2):
            return self._lca(root, node1, node2)
        
        return None
    
    def lcaWithoutCheckNodeInTree(self, root, node1, node2):
        result = self._lcaWithoutCheckNodeInTree(root, node1, node2)
        if result is not None and result.isAncestor:
            return result.node
        
        return None
        
    def _isNodeInTree(self, root, node):
        if root is None:
            return False
        
        if root != node:
            return self._isNodeInTree(root.left, node) or self._isNodeInTree(root.right, node)
        
        return True
    
    def _lca(self, root, node1, node2):
        if root is None:
            return None

        # since node1 and node2 are definitely in the tree, and we touched the most-bottom,
        # so only 2 cases here: LCA is in the path searching down, or LCA is this very root,
        # leave the judgement later(*)
        if root is node1 or root is node2:
            return root
        
        # we not touched the bottom yet, so continue searching down.
        leftNode = self._lca(root.left, node1, node2)
        rightNode = self._lca(root.right, node1, node2)
        
        # (*) continue, in this very root, left and right both returned nodes(of cause different nodes).
        # this very root is the LCA
        if leftNode is not None and rightNode is not None:
            return root
        
        # only one of the search returned 1 node, this node is the LCA, the other node is the child of this
        # returned node
        
        return leftNode if leftNode is not None else rightNode
    
    def _lcaWithoutCheckNodeInTree(self, root, node1, node2):
        if root is None:
            return LCAResult(None, isAncestor = False)
        
        if root is node1 and root is node2:
            # since we don't pre-check if node is in the tree, so if we reached bottom, we have 3 situations:
            # 1. one ancestor of this root is the LCA
            # 2. this root is the LCA
            # 3. the other node is not in the tree
            # only when node1 == node2 == root, can we make sure the root is the LCA
            return LCAResult(root, isAncestor = True)
        
        # we not touched the bottom yet, so continue searching down.
        leftResult = self._lcaWithoutCheckNodeInTree(root.left, node1, node2)
        rightResult = self._lcaWithoutCheckNodeInTree(root.right, node1, node2)
        
        # each result has 3 possible results:
        # left: None, isAncestor True, isAncestor False
        # right: None, isAncestor True, isAncestor False
        
        if leftResult.isAncestor:
            return leftResult
        
        if rightResult.isAncestor:
            return rightResult
        
        # only when left and right result both have node, can we make sure that this root
        # is an ancestor
        if leftResult.node is not None and rightResult.node is not None:
            return LCAResult(root, isAncestor = True)
        
        # both left and right touched bottom
        if root is node1 or root is node2:
            # only when root is one of the node and we sure find another node in the child trees,
            # can we make sure that this node is an ancestor
            isAncestor = leftResult.node is not None or rightResult.node is not None
            return LCAResult(root, isAncestor)
        
        return LCAResult(
            leftResult.node if leftResult.node is not None else rightResult.node,
            isAncestor = False
        )
                

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [88]:
# %load test_lca.py
from nose.tools import assert_equal


class TestLowestCommonAncestor(object):

    def test_lca(self):
        node10 = Node(10)
        node5 = Node(5)
        node12 = Node(12)
        node3 = Node(3)
        node1 = Node(1)
        node8 = Node(8)
        node9 = Node(9)
        node18 = Node(18)
        node20 = Node(20)
        node40 = Node(40)
        node3.left = node1
        node3.right = node8
        node5.left = node12
        node5.right = node3
        node20.left = node40
        node9.left = node18
        node9.right = node20
        node10.left = node5
        node10.right = node9
        root = node10
        node0 = Node(0)
        binary_tree = BinaryTree()
        assert_equal(binary_tree.lca(root, node0, node5), None)
        assert_equal(binary_tree.lca(root, node5, node0), None)
        assert_equal(binary_tree.lca(root, node1, node8), node3)
        assert_equal(binary_tree.lca(root, node12, node8), node5)
        assert_equal(binary_tree.lca(root, node12, node40), node10)
        assert_equal(binary_tree.lca(root, node9, node20), node9)
        assert_equal(binary_tree.lca(root, node3, node5), node5)
        print('Success: test_lca')


def main():
    test = TestLowestCommonAncestor()
    test.test_lca()


if __name__ == '__main__':
    main()

Success: test_lca


## Solution Notebook

Review the [Solution Notebook]() for a discussion on algorithms and code solutions.